loading files

In [ ]:
import pandas
import matplotlib.pyplot as plt

In [ ]:
path_to_snp_statistics = 'input/SNP_statistics_file.txt'
path_to_snp_call_contrast_positions = 'input/SNP_call_contrast_positions_file.txt'
path_to_cpf_csv = 'input/cpf.csv'

In [ ]:
snp_statistics_df = pandas.read_csv(path_to_snp_statistics, delimiter='\t', header=0)
snp_call_contrast_positions_df = pandas.read_csv(path_to_snp_call_contrast_positions, delimiter='\t', skiprows=5, header=0)

In [ ]:
snp_call_contrast_positions_df

In [ ]:
snp_statistics_df

In [ ]:
for column_name in snp_call_contrast_positions_df.columns:
    if column_name == 'probeset_id':
        pass
    else:
        name_list = column_name.split('.')
        if name_list[-1] == 'CEL_log_ratio':
            #print('preimenuj')
            #print(name_list[0])
            snp_call_contrast_positions_df.rename(columns={column_name: name_list[0]}, inplace=True)
        else:
            #print('ukloni')
            snp_call_contrast_positions_df.drop(column_name, axis=1, inplace=True)

In [ ]:
snp_call_contrast_positions_df

In [ ]:
#ovaj dio mozda i ne treba
#snp_call_contrast_positions_df.to_csv(path_to_cpf_csv, index=False)

In [ ]:
demo_df = pandas.merge(snp_call_contrast_positions_df, snp_statistics_df, on=['probeset_id'], how='inner')


In [ ]:
#demo_df

In [ ]:
#print(demo_df.shape)
#print(snp_call_contrast_positions_df.shape)
#print(snp_statistics_df.shape)

In [ ]:
for index, row in demo_df.iterrows():
    if row['n_NC'] >= 20:
        demo_df.drop(index, inplace=True)
demo_df.reset_index(drop=True, inplace=True)

In [ ]:
#print(demo_df.shape)

In [ ]:
aa_meanx_min_value = demo_df['AA.meanX'].min()
bb_meanx_min_value = demo_df['BB.meanX'].min()
aa_meanx_max_value = demo_df['AA.meanX'].max()
bb_meanx_max_value = demo_df['BB.meanX'].max()
aa_meanx_delta = abs(aa_meanx_max_value) - abs(aa_meanx_min_value)
bb_meanx_delta = abs(bb_meanx_max_value) - abs(bb_meanx_min_value)

aa_bb = (aa_meanx_delta + bb_meanx_delta) / 2

In [ ]:
print(aa_meanx_min_value)
print(bb_meanx_min_value)
print(aa_meanx_max_value)
print(bb_meanx_max_value)
print(aa_meanx_delta)
print(bb_meanx_delta)
print(aa_bb)


In [ ]:
for index, row in demo_df.iterrows():
    demo_df.at[index, 'AA-BB'] = demo_df.at[index, 'AA.meanX'] - demo_df.at[index, 'BB.meanX']
    if demo_df.at[index, 'AA-BB'] < aa_bb:
        demo_df.drop(index, inplace=True)
demo_df.reset_index(drop=True, inplace=True)

In [ ]:
#demo_df

In [ ]:
columns_to_remove = ['n_NC', 'AA.meanX', 'AA.varX', 'AB.meanX', 'BB.meanX', 'BB.varX', 'AA-BB']
demo_df.drop(columns=columns_to_remove, inplace=True)

In [ ]:
demo_df

In [ ]:
columns_to_remove = ['probeset_id']
demo_df.drop(columns=columns_to_remove, inplace=True)

In [ ]:
demo_df

In [ ]:
output_directory = 'histograms'
import os
os.makedirs(output_directory, exist_ok=True)
for column in demo_df.columns:
    plt.hist(demo_df[column], bins=1000, edgecolor='black')
    plt.xlabel(column)
    plt.ylabel('Frequency')
    plt.title(f'Histogram for {column}')
    output_file_path = os.path.join(output_directory, f'{column}_histogram.png')
    plt.savefig(output_file_path, bbox_inches='tight')
    #plt.show()
    plt.clf()
plt.close('all')